## Text classification using pre-trained models
BERT case (bidirectional encoder representations from transformers)

In [ ]:
# ktrain is a library that allows to:
# - build models
# - estimate optimal learning rate
# - loading and preprocessing text and image data from various sources
!pip install ktrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 48.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 75.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 653.6/653.6 kB 59.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 115.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 48.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━

In [ ]:
import ktrain
from ktrain import text

In [ ]:
import os

from google.colab import files
import tensorflow as tf

dataset = tf.keras.utils.get_file(
  fname="aclImdb.tar.gz",
  origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz",
  extract=True,
)
IMDB_DATADIR = os.path.join(os.path.dirname(dataset), "aclImdb")

84125825/84125825 [==============================] - 8s 0us/step


In [ ]:
# ktrain method texts_from_folder allows to split the dataset into train and test
# data from the specified folder and preprocess it based on the BERT model.
# In addition, BERT model and build requirements are also downloaded
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_folder(
  IMDB_DATADIR,
  maxlen=500,
  preprocess_mode='bert',
  train_test_names=['train', 'test'],
  classes=['pos', 'neg']
  )

detected encoding: utf-8
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [ ]:
# text_classifier builds and returns a text regression model
# get_learner returns a learner instance that can be used to tune and train Keras models.
model = text.text_classifier('bert', (x_train, y_train), preproc=preproc)
learner = ktrain.get_learner(model,train_data=(x_train, y_train), val_data=(x_test, y_test), batch_size=6)

Is Multi-Label? False
maxlen is 500


/usr/local/lib/python3.10/dist-packages/keras/initializers/initializers.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


done.


In [ ]:
# trains and tunes the model's parameters
learner.fit_onecycle(2e-5, 4)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/4
4167/4167 [==============================] - ETA: 0s - loss: 0.2620 - accuracy: 0.8911

In [ ]:
# https://towardsdatascience.com/bert-text-classification-in-3-lines-of-code-using-keras-264db7e7a358

model.save